# 🧠 Alzheimer MRI Sınıflandırması: Transfer Öğrenme ve Optimizasyon

Bu notebook, temel bir CNN modelinin sınırlılıklarını aşarak, transfer öğrenme ve gelişmiş optimizasyon teknikleriyle modelimizin performansını nasıl zirveye taşıdığımızı belgelemektedir. Bu aşama, projemizin en önemli dönüm noktasıdır.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
import sys

# Proje kök dizinini Python yoluna ekle
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

from src.model import get_model
from src.dataset import build_dataloaders
from src.transforms import base_train_transform, default_transform
from src.losses import FocalLoss
from src.evaluate import evaluate_model
from src.visualization import plot_training, save_confusion_matrix, save_classification_report

# Not: Bu notebook'ta eğitim döngüsü tekrarlanmayacaktır.
# Yalnızca stratejiler ve sonuçlar anlatılacaktır.

### 1. Transfer Öğrenme Modeli (İlk Deneme)

Baseline modelinin aksine, bu aşamada **ImageNet** üzerinde önceden eğitilmiş bir **ResNet50** mimarisi kullanılmıştır. İlk denemede, modelin sadece son katmanları eğitilmiştir (`fine-tuning` yok).

**Giriş:**
* **`use_sampler=True`**: Sınıf dengesizliği sorununu çözmek için `WeightedRandomSampler` kullanıldı.
* **`use_focal=False`**: Kayıp fonksiyonu olarak `CrossEntropyLoss` kullanıldı.

**Sonuçlar:**

Bu strateji, modelin bir önceki plato seviyesini aşmasını sağladı, ancak hala `VeryMildDemented` sınıfında zorlanıyordu.

* **Genel Doğruluk:** %73
* **`VeryMildDemented` F1-Score:** %38

**Çıkarım:**
Model, `ModerateDemented` ve `NonDemented` sınıflarını yüksek doğrulukla ayırt edebilirken, `VeryMildDemented` sınıfı için hala zorlanıyordu. Bu, modelin genelleme yeteneğinin artırılması gerektiğini gösteriyordu.

### 2. Final Model (İyileştirilmiş Strateji)

Modelin `VeryMildDemented` sınıfındaki performansını artırmak ve genel genelleme yeteneğini maksimize etmek için birleşik bir strateji uygulandı.

**Uygulanan İyileştirmeler:**
1.  **Fine-tuning:** **ResNet50'nin tüm katmanları (`freeze_backbone=False`)** yeniden eğitime açıldı.
2.  **`L2 Regularization`:** Aşırı öğrenmeyi önlemek için `Adam` optimizer'a `weight_decay=1e-4` parametresi eklendi.
3.  **Düşük Öğrenme Oranı (`lr=1e-4`):** Önceden eğitilmiş ağırlıkları korumak için öğrenme oranı düşürüldü.
4.  **Birleşik Dengeleme:** **`WeightedRandomSampler`** ve **`Focal Loss`** stratejileri birleştirilerek, modelin `VeryMildDemented` gibi zor sınıflardaki her hatasını daha ağır bir şekilde cezalandırması sağlandı.

**Sonuçlar:**

Bu birleşik strateji, modelin performansında dramatik bir sıçrama yarattı.



* **Genel Doğruluk:** %97.47
* **Sınıf Bazlı F1-Skorları:**
    * `MildDemented`: %98
    * `ModerateDemented`: %100 🏆
    * `NonDemented`: %97
    * `VeryMildDemented`: %97

#### **Çıkarım:**
* `Training` ve `Validation` eğrileri arasındaki makas kapanmıştır.
* `Validation Accuracy` %97'lere ulaşarak modelin genelleme yeteneğinin zirveye ulaştığı kanıtlanmıştır.
* **`VeryMildDemented`** sınıfı için elde edilen **%97'lik F1-score**, en zorlu problemin başarıyla çözüldüğünü gösterir.
* `Confusion Matrix` ve `Classification Report`'a göre, model tüm sınıflar için dengeli ve güvenilir bir performans sergilemektedir.